In [1]:
import collections
import math
import torch
import torch.nn as nn
from d2l import torch as d2l

In [2]:
class Seq2SeqEncoder(d2l.Encoder):
    """用于序列到序列学习的循环神经网络编码器"""
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                dropout=0, **kwargs):
        super(Seq2SeqEncoder, self).__init__(**kwargs)
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embed_size)
        # Recurrent layer
        self.rnn = nn.LSTM(embed_size, num_hiddens, num_layers,
                            dropout=dropout)

    def forward(self, X, *args):
        # The output `X` shape: (`batch_size`, `num_steps`, `num_hiddens`)
        # 这里因为Seq2Seq的输出是Decoder的输入，并没有经过最后的输出层，所以最后一个维度是num_hiddens
        X = self.embedding(X)
        X = X.permute(1, 0, 2) # permute(dims)将tensor的维度换位。
        # 一般RNN输入就是时间步，批量大小，词表大小
        # LSTM输入就是时间步，批量大小，词表大小
        # RNN输出就是时间步，批量大小，隐藏单元大小
        # LSTM输出就是时间步，批量大小，隐藏单元大小
        output, (state_h, state_c) = self.rnn(X)
        return output, (state_h, state_c)

In [3]:
encoder = Seq2SeqEncoder(vocab_size=10, embed_size=8, num_hiddens=16, num_layers=2)
encoder.eval() #Dropout不会生效
X = torch.zeros((4, 7), dtype=torch.long) # 4是batch_size, 7是句子的长度
output, state = encoder(X)
output.shape # torch.Size([7, 4, 16])

torch.Size([7, 4, 16])

In [4]:
# 2层，batch_size=4，num_hiddens=16
state[0].shape # torch.Size([2, 4, 16])

torch.Size([2, 4, 16])

In [5]:
class Seq2SeqDecoder(d2l.Decoder):
    """用于序列到序列学习的循环神经网络解码器"""
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 dropout=0, **kwargs):
        super(Seq2SeqDecoder, self).__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.rnn = nn.LSTM(embed_size + num_hiddens, num_hiddens, num_layers,
                          dropout=dropout)
        # 输出层
        self.dense = nn.Linear(num_hiddens, vocab_size)

    def init_state(self, enc_outputs, *args):
        return enc_outputs[1] # enc_outputs[1]是encoder的state

    def forward(self, X, state):
        # 输出'X'的形状：(batch_size,num_steps,embed_size)
        X = self.embedding(X).permute(1, 0, 2)
        # 广播context，使其具有与X相同的num_steps
        # num_layers = 2，这里拿的是第二层rnn的隐藏状态
        """
         上下文操作。这里state[-1]拿到的是“最右上角的”H(这个H融合和所有的信息)如果state是【2，4，16】的，
         那state[-1]就是【1,4,16】的。repeat重复时间步次。这样，每一个时间步都可以用到最后的H信息，
         与新的输入X做concat操作（这也是为什么解码器的self.rnn是ebd_size + num_hiddens的原因）。
         如果state[-1]是【1,4,16】，时间步是7，那重复完之后就是【7,4,16】的（7个时间步，4是batch_size，16是state隐藏单元的个数）。
        """
        context = state[-1].repeat(X.shape[0], 1, 1) # context的形状：(batch_size,num_steps,num_hiddens)
        X_and_context = torch.cat((X, context), 2)
        output, state = self.rnn(X_and_context, state)
        output = self.dense(output).permute(1, 0, 2)
        # output的形状:(batch_size,num_steps,vocab_size)
        # state的形状:(num_layers,batch_size,num_hiddens)
        return output, state

In [7]:
decoder = Seq2SeqDecoder(vocab_size=10, embed_size=8, num_hiddens=16,
                         num_layers=2)
decoder.eval()
state = decoder.init_state(encoder(X))
output, state = decoder(X, state)
output.shape, state.shape

RuntimeError: Sizes of tensors must match except in dimension 2. Expected size 7 but got size 14 for tensor number 1 in the list.